In [4]:
import zipfile
import os
import io
import psycopg2
import requests

# Function to download zip file from a remote location
def download_zip(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)

# Retrieve the postgres connection string from secrets
from google.colab import userdata
connection_string = userdata.get('postgres_connection_string')

# Establish a connection to the database
conn = psycopg2.connect(connection_string)

# Create a cursor object to execute SQL queries
cur = conn.cursor()

# Path to the zip archive containing the dataset
zip_file_url = "https://github.com/dnuentsa/cloud-tools-for-analytics/raw/main/resources/postgresql/car_sharing_dataset.zip"

# Path to save the downloaded zip file
zip_file_path = "temp_zip_file.zip"

# Download the zip file from the remote location
download_zip(zip_file_url, zip_file_path)

# Extract SQL files from the zip archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Get list of files sorted alphabetically
    sql_files = sorted(zip_ref.namelist())
    print(sql_files)
    # Iterate through sorted list of files
    for filename in sql_files:
        if filename.endswith(".sql"):
            # Extract file
            zip_ref.extract(filename, "temp_sql_files")

            # Read SQL file
            file_path = os.path.join("temp_sql_files", filename)
            with open(file_path, "r") as file:
                sql_statements = file.read()

            # Execute SQL statements
            cur.execute(sql_statements)

# Commit changes to the database
conn.commit()

# Close cursor and connection
cur.close()
conn.close()
print("All Data have been inserted successfully in the remote database")